In [2]:
import os
import pandas as pd
import numpy as np
import argparse


# Summary:
# - Find married couples in the PEOPLA data and add relevant entries to PEO_REL data


# run_dir = "/media/michael/SSD3/Dropbox/workspace/2.research/01.Sources/01.Primary/01.Manuscripts/00.WWW/FS, FamilySearch/359031, Cairn of Lochwinyoch matters, 1827-37"
# change this to also take a command line argument...





In [ ]:
def get_spouse_rels( fpath ):
    d = pd.read_csv(fpath, "\t", header=0)

    print( d.src_ref )

    # dplyr		pandas equivalent
    # head		head
    # mutate	assign
    # select	filter
    # rename	rename
    # filter	query
    # arrange	sort_values
    # group_by	groupby
    # summarize	agg

    #print( d.columns )

    ### FILTER THE DATA FRAME TO ONLY FIND THOSE MARRIED RECORDS
    # https://stmorse.github.io/journal/tidyverse-style-pandas.html
    d_married = ( d.query( 'action=="MARRIED"' ) )

    ### GROUP THE MARRIAGES THAT ARE ON THE SAME LINE
    married_grouping = ( d_married.groupby( ['src_linenum'] ) )

    ### ADD A COLUMN TO IDENTIFY THE 1st/2nd (3rd?) PERSON IN EACH MARRIAGE
    d_married['person_number'] = married_grouping['src_linenum'].rank(method="first", na_option="top")

    ### PIVOT (equivalent of pivot_wider() in R)
    d_married_wide = d_married.pivot(index='src_linenum', columns='person_number', values='pers_id')

    print( "---------------------\n")

    return d_married_wide

    #TODO: return the new found spouse rels as a list
    #
    # 	PEO_REL example:
    #
    #	src_ref										src_linenum		pers1_id			pers1_id_context	rel		pers2_id			pers2_id_context
    #	01.01.01.00.FS.359031.1419516.02_TXT.0306	291				WILSON, Margaret	291					SPOUSE	WILSON, William		(4)

In [16]:
fpath = "/Users/lisahopcroft_tmp/Projects/History/dat/03_^DAT/^0010, --- ADAM of Bowfield [~].jpg.txt.PEOPLA.tsv"
d = pd.read_csv(fpath, "\t", header=0)

# dplyr		pandas equivalent
# head		head
# mutate	assign
# select	filter
# rename	rename
# filter	query
# arrange	sort_values
# group_by	groupby
# summarize	agg

#print( d.columns )

### FILTER THE DATA FRAME TO ONLY FIND THOSE MARRIED RECORDS
# https://stmorse.github.io/journal/tidyverse-style-pandas.html
d_married = ( d.query( 'action=="MARRIED"' ) )

### GROUP THE MARRIAGES THAT ARE ON THE SAME LINE
married_grouping = ( d_married.groupby( ['src_linenum'] ) )

### ADD A COLUMN TO IDENTIFY THE 1st/2nd (3rd?) PERSON IN EACH MARRIAGE
d_married['person_number'] = married_grouping['src_linenum'].rank(method="first", na_option="top")

### PIVOT (equivalent of pivot_wider() in R)
d_married_wide = d_married.pivot(index='src_linenum', columns='person_number', values='pers_id')

print( d_married_wide )

person_number                 1.0                 2.0
src_linenum                                          
124                  ADAM, Robert   KIRKWOOD, Elspeth
147                ADAM, Margaret    CALDWELL, Robert
162            CALDWELL, Margaret        HUNTER, John
188                CALDWELL, Mary  BARCLAY, Alexander
231                    ADAM, Nans       CLERK, Andrew
249                 CLERK, Robert                   .
262               CLERK, Margaret     GEMMILL, Thomas
273                  CLERK, Nanse        LATTA, James
286                    ADAM, Mary      LATTA, William
306                 LATTA, Robert      BRAEDINE, Jean
404                 ADAM, William      COCHRAN, James
474                  ADAM, Robert          ORR, Janet
547                 ADAM, William         EWING, Mary


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:

#print( d_married.columns )


print( d_married_wide.head )

print( "---------------------\n")

In [ ]:
if __name__ == "__main__":

    ### READ COMMAND LINE ARGUMENTS
    
    parser = argparse.ArgumentParser(description="Identify spouses in a PEO/PLA resource")

    parser.add_argument("run_dir",
                        nargs='?',
                        help="The directory in which to search for PEO/PLA data sources",
                        default=".")
    parser.add_argument("-v",
                        help="Print additional messages at run time",
                        action='store_true')

    args = parser.parse_args()

    is_debugging = False
    if args.v: is_debugging = True

    run_dir = args.run_dir

    error_msgs = []

    for root, dirs, files in os.walk( run_dir ):
        for fname in files:
            if fname.endswith(".PEOPLA.tsv"):
                spouse_rels = get_spouse_rels( "%s/%s" % ( root, fname ) )
                print( spouse_rels )
                
                #TODO: append these spouse rels to the relevant PEO_REL file